<a href="https://colab.research.google.com/github/UmeshikaThathsarani/FuelO/blob/feature%2Fmodel/FuelO_Finalized_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold, train_test_split
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset and preprocess the data
data = pd.read_csv('/content/Correlated - Copy.csv')
categorical_features = ['Year', 'Make', 'Transmission', 'Turbocharger', 'Fuel Type']
numerical_features = ['Engine Cylinders', 'MPG']
onehot_encoder = OneHotEncoder(sparse=False)
scaler = StandardScaler()
X_cat = onehot_encoder.fit_transform(data[categorical_features])
X_num = scaler.fit_transform(data[numerical_features])
X = np.concatenate((X_cat, X_num), axis=1)
y = data[['City Fuel Consumption', 'Highway Fuel Consumption']].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the DNN model architecture with L2 regularization
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dense(2, activation='linear')
])

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model using K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True)
history = {'loss': [], 'val_loss': []}
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    hist = model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=32, validation_data=(X_val_fold, y_val_fold))
    history['loss'].extend(hist.history['loss'])
    history['val_loss'].extend(hist.history['val_loss'])

# Evaluate the model performance on the testing data
y_pred = model.predict(X_test)
mse_city = mean_squared_error(y_test[:,0], y_pred[:,0])
mse_highway = mean_squared_error(y_test[:,1], y_pred[:,1])
r2_city = r2_score(y_test[:,0], y_pred[:,0])
r2_highway = r2_score(y_test[:,1], y_pred[:,1])
print("Mean squared error for city fuel consumption:", mse_city)
print("Mean squared error for highway fuel consumption:", mse_highway)
print("R-squared value for city fuel consumption:", r2_city)
print("R-squared value for highway fuel consumption:", r2_highway)

# Visualize the training and validation loss curves
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Epoch 1/10
819/819 [==============================] - 8s 7ms/step - loss: 2674898.7500 - val_loss: 56675.7500
Epoch 2/10
819/819 [==============================] - 4s 5ms/step - loss: 56375.5039 - val_loss: 55460.9531
Epoch 3/10
819/819 [==============================] - 4s 5ms/step - loss: 55701.9570 - val_loss: 55230.6016
Epoch 4/10
819/819 [==============================] - 2s 3ms/step - loss: 55682.5234 - val_loss: 55997.2930
Epoch 5/10
819/819 [==============================] - 2s 3ms/step - loss: 55527.8984 - val_loss: 55695.2734
Epoch 6/10
819/819 [==============================] - 2s 2ms/step - loss: 55161.2461 - val_loss: 54125.1797
Epoch 7/10
819/819 [==============================] - 2s 2ms/step - loss: 53044.5469 - val_loss: 49941.8945
Epoch 8/10
819/819 [==============================] - 2s 2ms/step - loss: 47018.1680 - val_loss: 42855.3359
Epoch 9/10
819/819 [==============================] - 2s 2ms/step - loss: 40061.9688 - val_loss: 41399.6133
Epoch 10/10
819/819 [=====